<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch 6. 여러 개의 값에 대한 조작

In [5]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 6.1 문자열을 연결하는 쿼리
데이터를 다양한 관점으로 바라보기 위해 용도에 맞게 여러 개의 데이터를 연결해서 전처리해주는 경우가 많습니다. 샘플 데이터를 사용하여 주소를 연결하는 쿼리를 작성해보겠습니다. 

샘플 데이터를 생성합니다.

In [7]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_user_location;
CREATE TABLE sqldata.mst_user_location(
    user_id   string
  , pref_name string
  , city_name string
);
INSERT INTO sqldata.mst_user_location
VALUES
    ('U001', '서울특별시', '강서구')
  , ('U002', '경기도수원시', '장안구'  )
  , ('U003', '제주특별자치도', '서귀포시')
;


""


In [9]:
%%bigquery --project mygcpproject-340112
SELECT 
  user_id
  , CONCAT(pref_name, city_name) AS pref_city
FROM
  sqldata.mst_user_location
;

,user_id,pref_city
0,U003,제주특별자치도서귀포시
1,U001,서울특별시강서구
2,U002,경기도수원시장안구


CONCAT 함수를 이용하여 문자열을 연결해주면 지정한 변수명인 pref_city에 연결된 주소가 출력되는 것을 확인할 수 있습니다. 

### 6.2 여러개의 값 비교하기

In [10]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.quarterly_sales;
CREATE TABLE sqldata.quarterly_sales (
    year integer
  , q1   integer
  , q2   integer
  , q3   integer
  , q4   integer
);

INSERT INTO sqldata.quarterly_sales
VALUES
    (2015, 82000, 83000, 78000, 83000)
  , (2016, 85000, 85000, 80000, 81000)
  , (2017, 92000, 81000, NULL , NULL )
;

""


우선 quarterly_sales라는 이름의 dataset을 만들어 줍니다. 

In [12]:
%%bigquery --project mygcpproject-340112
SELECT
    year
  , q1
  , q2
    -- Q1과 Q2 의 매출 변화 평가하기
  , CASE
    WHEN q1 < q2 THEN '+'
    WHEN q1 = q2 THEN '-'
    ELSE '-'
  END AS judge_q1_q2
    -- Q1과 Q2의 매출액 차이 계산하기
  , q2 - q1 AS diff_q2_q1
    -- Q1과 Q2의 매출변화를 1, 0, -1로 표현하기
  , SIGN(q2 - q1) AS sign_q2_q1
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,q1,q2,judge_q1_q2,diff_q2_q1,sign_q2_q1
0,2015,82000,83000,+,1000,1
1,2016,85000,85000,-,0,0
2,2017,92000,81000,-,-11000,-1


quarterly_sales 데이터에서 year, q1, q2 칼럼을 가져온 뒤 CASE 함수를 이용하여 조건을 작성하였습니다. 조건에 따른 결과를 judge_q1_q2 로 지정한 뒤에 매출액의 차이와 매출 변화를 출력합니다. SIGN 은 매개변수가 양수일 때 1, 음수일 때 -1을 출력하는 함수입니다. 

### 6.3 연간 최대/최소 4분기 매출 찾기


In [14]:
%%bigquery --project mygcpproject-340112
SELECT
    year
    -- Q1~Q4의 최대 매출 구하기
  , greatest(q1, q2, q3, q4) AS greatest_sales
    -- Q1~Q4의 최소 매출 구하기
  , least(q1, q2, q3, q4) AS least_sales
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,greatest_sales,least_sales
0,2015,83000.0,78000.0
1,2016,85000.0,80000.0
2,2017,NaN,NaN
